In [2]:
# %% imports and setting up
from os import chdir as setwd 
from os import getcwd as getcwd

# setting working directory
user = "TS"  # <-- type your initials here
work_dir = ""

# add the file path to the repo on your machine
if user == "TS":
    work_dir = "C:/Users/t-sut/Documents/Monash/Monash2022/Sem2/FIT3164/" + \
                "ds08_renewable_energy_scheduling/"
elif user == "JK":
    work_dir = "C:/Users/jjaso/Documents/ds08_renewable_energy_scheduling/"
elif user == "JS": 
    work_dir = "/Users/jasonnathaniel/Documents/Monash University/Sem 2 2022/" + \
                "FIT3164 Data science project 2/ds08_renewable_energy_scheduling/"

setwd(work_dir)
print("Current working directory: ", getcwd())

# import data science libraries
import matplotlib.pyplot as plt
import pandas as pd
import util_data_loader as dl
import util_data_cleaning as clean
from util_MASE import mase 
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

Current working directory:  C:\Users\t-sut\Documents\Monash\Monash2022\Sem2\FIT3164\ds08_renewable_energy_scheduling


# Read, clean and reshape data

In [17]:
# %% Read and clean power data
# using phase 2 data as it's the same as phase 1, but for more time stamps
powr = dl.convert_tsf_to_dataframe("rawdata/nov_data.tsf")[0]

# clean and reshape
powr = clean.extract_data(powr)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


In [4]:
powr

,Building0,Building1,Building3,Building4,Building5,Building6,Solar0,Solar1,Solar2,Solar3,Solar4,Solar5,timestamp
0,153.500000,9.900000,249.0,0.78195,4.672332,35.599998,0.00,7.63,3.75,1.34,1.66,0.00,2020-04-25 14:00:00
1,149.899994,2.700000,248.0,1.00000,4.672332,36.400002,0.00,7.63,3.75,1.34,1.66,0.00,2020-04-25 14:15:00
2,149.899994,4.900000,248.0,0.78195,4.672332,36.400002,0.00,7.63,3.75,1.34,1.66,0.00,2020-04-25 14:30:00
3,149.899994,7.800000,248.0,1.00000,4.672332,33.000000,0.00,7.63,3.75,1.34,1.66,0.00,2020-04-25 14:45:00
4,149.899994,10.300000,248.0,0.78195,4.672332,37.000000,0.00,7.63,3.75,1.34,1.66,0.00,2020-04-25 15:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21059,218.600006,17.299999,554.0,1.00000,18.000000,40.000000,13.21,2.32,2.21,3.11,1.31,9.86,2020-11-30 22:45:00
21060,218.600006,23.100000,554.0,1.00000,36.000000,46.400002,8.24,2.21,2.10,1.97,1.25,6.22,2020-11-30 23:00:00
21061,207.199997,6.600000,694.0,2.00000,18.000000,46.400002,9.42,2.00,1.90,2.24,1.13,7.14,2020-11-30 23:15:00
21062,207.199997,11.000000,694.0,1.00000,18.000000,44.599998,7.63,1.78,1.70,1.78,1.01,5.64,2020-11-30 23:30:00


# Model

In [5]:
# set params
horizon = 1

In [6]:
offset = pd.Timedelta(minutes = 15 * horizon)
lower_timestamp = pd.to_datetime("2020-11-01 00:00:00", format = "%Y-%m-%d %H:%M:%S") - offset
upper_timestamp = pd.to_datetime("2020-11-30 23:45:00", format = "%Y-%m-%d %H:%M:%S") - offset

preds = powr.loc[(lower_timestamp <= powr.timestamp) & (upper_timestamp >= powr.timestamp), :]
preds.timestamp = preds.timestamp + offset

preds

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,Building0,Building1,Building3,Building4,Building5,Building6,Solar0,Solar1,Solar2,Solar3,Solar4,Solar5,timestamp
18183,77.599998,7.800000,327.0,1.00000,4.672332,27.400000,28.230000,7.34,7.30,6.76,4.49,20.639999,2020-11-01 00:00:00
18184,77.599998,10.100000,327.0,1.00000,4.672332,27.600000,31.900000,7.95,8.26,7.69,5.13,23.010000,2020-11-01 00:15:00
18185,25.700001,3.100000,315.0,0.78195,4.672332,27.600000,32.020000,8.51,8.11,7.52,5.01,23.730000,2020-11-01 00:30:00
18186,25.700001,5.300000,315.0,0.78195,4.672332,25.600000,32.139999,9.07,7.95,7.35,4.89,24.450001,2020-11-01 00:45:00
18187,25.700001,7.900000,315.0,1.00000,4.672332,25.600000,36.680000,9.38,9.12,8.15,5.59,28.049999,2020-11-01 01:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21058,218.600006,11.500000,554.0,2.00000,36.000000,40.000000,9.040000,2.42,2.31,2.15,1.36,6.820000,2020-11-30 22:45:00
21059,218.600006,17.299999,554.0,1.00000,18.000000,40.000000,13.210000,2.32,2.21,3.11,1.31,9.860000,2020-11-30 23:00:00
21060,218.600006,23.100000,554.0,1.00000,36.000000,46.400002,8.240000,2.21,2.10,1.97,1.25,6.220000,2020-11-30 23:15:00
21061,207.199997,6.600000,694.0,2.00000,18.000000,46.400002,9.420000,2.00,1.90,2.24,1.13,7.140000,2020-11-30 23:30:00


In [7]:
# doing this as the timestamp is sorted
offset = pd.Timedelta(minutes = 15 * horizon)
preds = powr.iloc[-(2880+horizon+1):-horizon].copy()
preds.timestamp = preds.timestamp + offset
preds

,Building0,Building1,Building3,Building4,Building5,Building6,Solar0,Solar1,Solar2,Solar3,Solar4,Solar5,timestamp
18182,77.599998,5.100000,327.0,1.00000,4.672332,27.400000,24.559999,6.73,6.35,5.83,3.85,18.290001,2020-10-31 23:45:00
18183,77.599998,7.800000,327.0,1.00000,4.672332,27.400000,28.230000,7.34,7.30,6.76,4.49,20.639999,2020-11-01 00:00:00
18184,77.599998,10.100000,327.0,1.00000,4.672332,27.600000,31.900000,7.95,8.26,7.69,5.13,23.010000,2020-11-01 00:15:00
18185,25.700001,3.100000,315.0,0.78195,4.672332,27.600000,32.020000,8.51,8.11,7.52,5.01,23.730000,2020-11-01 00:30:00
18186,25.700001,5.300000,315.0,0.78195,4.672332,25.600000,32.139999,9.07,7.95,7.35,4.89,24.450001,2020-11-01 00:45:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21058,218.600006,11.500000,554.0,2.00000,36.000000,40.000000,9.040000,2.42,2.31,2.15,1.36,6.820000,2020-11-30 22:45:00
21059,218.600006,17.299999,554.0,1.00000,18.000000,40.000000,13.210000,2.32,2.21,3.11,1.31,9.860000,2020-11-30 23:00:00
21060,218.600006,23.100000,554.0,1.00000,36.000000,46.400002,8.240000,2.21,2.10,1.97,1.25,6.220000,2020-11-30 23:15:00
21061,207.199997,6.600000,694.0,2.00000,18.000000,46.400002,9.420000,2.00,1.90,2.24,1.13,7.140000,2020-11-30 23:30:00


# Evaluate on test set

In [8]:
eval_data = powr.drop(["timestamp"], axis=1)
for i in range(eval_data.shape[1]): # for each col
    col_name = preds.columns[i]
    err = mase(np.array(preds)[:, i], np.array(eval_data)[-preds.shape[0]:, i], 2688)
    print(f"MASE of {col_name}: {err}")

MASE of Building0: 0.08081182762322704
MASE of Building1: 3.3354035740690553
MASE of Building3: 0.2696179421784682
MASE of Building4: 0.9207447772358267
MASE of Building5: 1.7352063188979547
MASE of Building6: 0.46054304034886245
MASE of Solar0: 0.3500164339588172
MASE of Solar1: 0.32778115436037
MASE of Solar2: 0.35226438341039945
MASE of Solar3: 0.3870147849839196
MASE of Solar4: 0.31151127909667387
MASE of Solar5: 0.3879825920937169


## Best horizon value by time series:

| TS | Building0 | Building1 | Building3 | Building4 | Building5 | Building6 | Solar0 | Solar1 | Solar2 | Solar3 | Solar4 | Solar5 | 
| ----- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |  
| Horizon | 1 | 8 | 1 | 2 | 2 | 1 | 1 | 1 | 1 | 1 | 1 | 1 |

Next step: Figure out how to incorporate different horizons into the naive model

In [9]:
# powr = dl.convert_tsf_to_dataframe("rawdata/nov_data.tsf")[0]
# temp = powr
# for row in range(temp.shape[0]):
#     temp.at[row, "series_value"] = temp.loc[row, "series_value"][:5]

In [20]:
# doing this as the timestamp is sorted
b1 = 4
b4 = 2
b5 = 2
horizons = [1, b1, 1, b4, b5, 1, 1, 1, 1, 1, 1, 1]
preds = np.zeros((2881, powr.shape[1]))
for col in range(eval_data.shape[1]):
    horizon = horizons[col]
    offset = pd.Timedelta(minutes = 15 * horizon)
    preds[:, col] = powr.iloc[-(2880+horizon+1):-horizon, col].copy()
#     preds.timestamp = preds.timestamp + offset
preds

array([[ 77.59999847,   8.39999962, 327.        , ...,   3.8499999 ,
         18.29000092,   0.        ],
       [ 77.59999847,  11.30000019, 327.        , ...,   4.48999977,
         20.63999939,   0.        ],
       [ 77.59999847,   2.29999995, 327.        , ...,   5.13000011,
         23.01000023,   0.        ],
       ...,
       [218.6000061 ,   5.0999999 , 554.        , ...,   1.25      ,
          6.21999979,   0.        ],
       [207.19999695,  11.5       , 694.        , ...,   1.13      ,
          7.13999987,   0.        ],
       [207.19999695,  17.29999924, 694.        , ...,   1.00999999,
          5.63999987,   0.        ]])

In [23]:
eval_data = powr.drop(["timestamp"], axis=1)
for i in range(eval_data.shape[1]): # for each col
    col_name = eval_data.columns[i]
    err = mase(preds[:, i], np.array(eval_data)[-preds.shape[0]:, i], 2688)
    print(f"MASE of {col_name}: {err}")

MASE of Building0: 0.08081182762322704
MASE of Building1: 0.7330738532458922
MASE of Building3: 0.2696179421784682
MASE of Building4: 0.7408192992360485
MASE of Building5: 0.8518995086918465
MASE of Building6: 0.46054304034886245
MASE of Solar0: 0.3500164339588172
MASE of Solar1: 0.32778115436037
MASE of Solar2: 0.35226438341039945
MASE of Solar3: 0.3870147849839196
MASE of Solar4: 0.31151127909667387
MASE of Solar5: 0.3879825920937169


In [ ]:
for row in range(temp.shape[0]):
    powr.at[row, "start_timestamp"] = pd.to_datetime("2020-04-25 14:00:00", format = "%Y-%m-%d %H:%M:%S")

In [ ]:
clean.extract_data(powr)

In [11]:
for row in range(powr.shape[0]):
    print(len(powr.series_value[row]))

5
5
5
5
5
5
5
5
5
5
5
5


In [12]:
powr

,series_name,start_timestamp,series_value
0,Building0,2016-07-03 21:30:00,"[283.8, 283.8, 283.8, 606.0, 606.0]"
1,Building1,2019-01-09 23:15:00,"[8.1, 15.7, 22.8, 32.7, 8.1]"
2,Building3,2016-03-01 04:15:00,"[1321.0, 1321.0, 1321.0, 1321.0, 1293.0]"
3,Building4,2019-07-03 04:45:00,"[2.0, NaN, 1.0, 2.0, NaN]"
4,Building5,2019-07-25 23:00:00,"[30.0, 31.0, 24.0, 34.0, 30.0]"
5,Building6,2019-07-25 01:45:00,"[36.8, 34.6, 34.6, 36.2, 36.2]"
6,Solar0,2020-04-25 14:00:00,"[0.0, 0.0, 0.0, 0.0, 0.0]"
7,Solar1,2018-12-31 13:00:00,"[0.0, 0.0, 0.0, 0.0, 0.0]"
8,Solar2,2019-06-05 14:00:00,"[0.0, 0.0, 0.0, 0.0, 0.0]"
9,Solar3,2019-06-05 14:00:00,"[0.0, 0.0, 0.0, 0.0, 0.0]"


## Export forecast as csv

In [13]:
preds

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       ...,
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [15]:
export_preds = preds[1:,:12]
export_preds = export_preds.transpose()

In [16]:
export_preds

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
export_preds.to_csv("forecasting/saved_forecasts/forecast_naive.csv", header = None)

In [25]:
powr.columns[:-1]

Index(['Building0', 'Building1', 'Building3', 'Building4', 'Building5',
       'Building6', 'Solar0', 'Solar1', 'Solar2', 'Solar3', 'Solar4',
       'Solar5'],
      dtype='object')